# doctor analysis

의사가 경험적 진단시 결과에 대한 계산

In [46]:
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import os
from glob import glob
import sys

import scipy.stats as stats # qqplot에 사용
import prince #MCA

In [47]:
#시각화 패키지들
import seaborn as sns
import matplotlib.pyplot as plt
%matplotlib inline
from matplotlib import font_manager, rc
font_path = "C:/Windows/Fonts/NGULIM.TTF"
font = font_manager.FontProperties(fname=font_path).get_name()
rc('font', family=font)
from statsmodels.graphics.mosaicplot import mosaic

plt.style.use("ggplot")


In [48]:
#분석 관련
from scipy.stats import chi2_contingency
from scipy.stats import kstest
from statsmodels.formula.api import ols
from factor_analyzer import FactorAnalyzer

In [49]:
X = pd.read_excel("../data/CIP_data.xlsx")
print(X.shape)
X.head()

(550, 47)


,date,ID,sex,age,Age,EFF,App,Ab,mc,iSBP,...,Leukopenia,WBC,Hb,Plt,CRP,Glu,BUN,Cr,Bacteria,CIP
0,2020-01-01,158092,0,1,67,0,1.0,1.0,0,71,...,0,11850,12.5,159,14.30,154,37.2,1.62,0,0
1,2020-01-01,1953490,0,1,72,0,1.0,1.0,1,133,...,0,10460,13.0,201,4.19,159,18.4,0.80,0,0
2,2020-01-01,1953526,0,1,74,0,1.0,1.0,0,172,...,0,8430,12.7,183,9.24,269,13.0,0.70,0,0
3,2020-01-01,1260373,1,1,78,0,1.0,2.0,0,166,...,0,5740,11.9,152,4.59,138,29.5,1.73,0,1
4,2020-01-02,1842207,0,0,48,0,1.0,1.0,0,122,...,0,10530,12.3,207,0.18,109,11.5,0.97,0,0


In [50]:
X.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 550 entries, 0 to 549
Data columns (total 47 columns):
 #   Column       Non-Null Count  Dtype  
---  ------       --------------  -----  
 0   date         550 non-null    object 
 1   ID           550 non-null    int64  
 2   sex          550 non-null    int64  
 3   age          550 non-null    int64  
 4   Age          550 non-null    int64  
 5   EFF          550 non-null    int64  
 6   App          361 non-null    float64
 7   Ab           546 non-null    float64
 8   mc           550 non-null    int64  
 9   iSBP         550 non-null    int64  
 10  iDBP         550 non-null    int64  
 11  iPR          550 non-null    int64  
 12  iRR          550 non-null    int64  
 13  iBT          550 non-null    float64
 14  hypothermia  550 non-null    int64  
 15  iSpO         530 non-null    float64
 16  Qsofa        550 non-null    int64  
 17  bed          550 non-null    int64  
 18  Foley        550 non-null    int64  
 19  Hsp-acq 

In [51]:
X[['sex','age','EFF','App','Ab','mc','hypothermia','Qsofa','bed','Foley',
   'Hsp-acq','Nursing','3m-Ab','3m-Adm','1y-UTI','PPIantacid',
   'CV','CRF','total malig','NR','Leukopenia','Bacteria','DM','HTN']] =X[['sex','age','EFF','App','Ab','mc','hypothermia','Qsofa','bed','Foley',
   'Hsp-acq','Nursing','3m-Ab','3m-Adm','1y-UTI','PPIantacid',
   'CV','CRF','total malig','NR','Leukopenia','Bacteria','DM','HTN']].astype('category') 

In [52]:
Ab_data = X.copy()

In [53]:
X.drop(['date','ID','age','EFF','App','Ab','hypothermia','Bacteria'],axis=1,inplace=True)

In [54]:

new_data=X.dropna(axis=0)



print("전체 데이터셋 사이즈 : ",new_data.shape)

전체 데이터셋 사이즈 :  (468, 39)


In [55]:
new_data.index

Int64Index([  0,   1,   2,   3,   4,   6,   7,   8,   9,  10,
            ...
            540, 541, 542, 543, 544, 545, 546, 547, 548, 549],
           dtype='int64', length=468)

In [56]:
Ab_data=Ab_data.loc[new_data.index]

In [57]:
Ab_data

,date,ID,sex,age,Age,EFF,App,Ab,mc,iSBP,...,Leukopenia,WBC,Hb,Plt,CRP,Glu,BUN,Cr,Bacteria,CIP
0,2020-01-01,158092,0,1,67,0,1.0,1.0,0,71,...,0,11850,12.5,159,14.30,154,37.2,1.62,0,0
1,2020-01-01,1953490,0,1,72,0,1.0,1.0,1,133,...,0,10460,13.0,201,4.19,159,18.4,0.80,0,0
2,2020-01-01,1953526,0,1,74,0,1.0,1.0,0,172,...,0,8430,12.7,183,9.24,269,13.0,0.70,0,0
3,2020-01-01,1260373,1,1,78,0,1.0,2.0,0,166,...,0,5740,11.9,152,4.59,138,29.5,1.73,0,1
4,2020-01-02,1842207,0,0,48,0,1.0,1.0,0,122,...,0,10530,12.3,207,0.18,109,11.5,0.97,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
545,2021-06-25,322916,0,1,74,0,1.0,1.0,0,135,...,0,20770,11.9,276,20.72,157,11.4,0.55,0,0
546,2021-06-27,786350,1,1,83,1,NaN,1.0,0,167,...,0,7850,11.1,84,4.40,112,16.5,1.51,0,1
547,2021-06-28,32925,0,1,77,1,NaN,1.0,0,81,...,0,10600,10.0,296,7.74,141,21.2,0.72,0,1
548,2021-06-29,174801,0,1,86,1,NaN,2.0,0,177,...,0,6560,7.3,209,0.84,115,34.0,2.68,0,1


In [58]:
Y=Ab_data['CIP']
Ab_data.drop(['CIP'],axis=1,inplace=True)

In [59]:
Ab_data = Ab_data[['Ab']]

In [60]:
from sklearn.model_selection import train_test_split
x_train, x_test, y_train, y_test = train_test_split(Ab_data, Y, stratify=Y, test_size=0.2, random_state=42)

In [61]:

print('train set : ',x_train.shape)
print('test set : ', x_test.shape)

train set :  (374, 1)
test set :  (94, 1)


In [62]:
print("test yes : ",sum(y_test==1))
print("test no : ",sum(y_test==0))

test yes :  47
test no :  47


In [68]:
y_test

253    0
317    1
235    1
439    0
220    0
      ..
451    0
136    0
259    1
286    0
91     0
Name: CIP, Length: 94, dtype: int64

In [71]:
(x_test==0)&(y_test==0)

,1,6,16,19,22,29,41,42,72,75,...,518,524,529,530,539,543,545,548,549,Ab
253,False,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False
317,False,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False
235,False,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False
439,False,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False
220,False,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
451,False,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False
136,False,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False
259,False,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False
286,False,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False


In [67]:
x_test

C:\Users\dlwhd\miniconda3\envs\localtorch\lib\site-packages\pandas\io\formats\format.py:1429: FutureWarning: Index.ravel returning ndarray is deprecated; in a future version this will return a view on self.
  for val, m in zip(values.ravel(), mask.ravel())


253    1.0
317    0.0
235    1.0
439    0.0
220    0.0
      ... 
451    1.0
136    1.0
259    2.0
286    0.0
91     1.0
Name: Ab, Length: 94, dtype: category
Categories (5, float64): [0.0, 1.0, 2.0, 3.0, 4.0]

In [94]:
from sklearn.metrics import confusion_matrix
from sklearn.metrics import accuracy_score
from sklearn.metrics import f1_score
from sklearn.metrics import auc
from sklearn.metrics import recall_score
from sklearn.metrics import precision_score


tp=0
fp=0
fn=0
tn=0

def doctor_cip_criterion(Ab,cip):
    Ab[Ab['Ab']!=0] = 1
    print("acc : ",accuracy_score(cip,Ab))
    print("f1_score : ",f1_score(cip,Ab))
    print("recall : ",recall_score(y_true=cip, y_pred=Ab, pos_label=1))
    print("specificity : ",recall_score(y_true=cip, y_pred=Ab, pos_label=0))
    print("precision : ",precision_score(y_true=cip, y_pred=Ab))
    
    matrix=confusion_matrix(cip,Ab)
    
    
    return matrix
    
sss=doctor_cip_criterion(x_test,y_test)

acc :  0.48936170212765956
f1_score :  0.6307692307692309
recall :  0.8723404255319149
specificity :  0.10638297872340426
precision :  0.4939759036144578


In [91]:
sss

array([[ 5, 42],
       [ 6, 41]], dtype=int64)